In [1]:
import warnings
warnings.filterwarnings('ignore')
import requests
import folium
from pandas.io.json import json_normalize

In [2]:
# 전국 또는 특정 지역의 스타벅스 매장 위치를 찾아서 지도위에 표시하기
# 1. requests 모듈로 스타벅스 매장 위치 데이터를 가져와서 딕셔너리 타입으로 변환한다.
# 2. 판다스의 json_normalize() 함수로 json이 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# 3. folium 모듈을 사용해 지도를 표시하고 지도위의 스타벅스 매장 위치에 마커를 표시한다.

In [3]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=OINSMG6A8S'
request = requests.post(targetSite, data={
    'ins_lat': 37.5108295,
    'ins_lng': 127.0292881,
    'p_sido_cd': '01',
    'p_gugun_cd': '', 
    'in_biz_cd': '',
    'iend': 1700,
    'set_date': ''
})
storeList = request.json()
print(type(storeList))
print(len(storeList['list']))

<class 'dict'>
567


In [4]:
# json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_normalize(데이터프레임으로 변환할 데이터가 저장된 딕셔너리, '딕셔너리의 key 중에서 데이터프레임으로 변환할 key')
star_df = json_normalize(storeList, 'list')
print(type(star_df))

<class 'pandas.core.frame.DataFrame'>


In [5]:
star_df.head()

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,t22,t21,p90,t05,t30,t36,t27,t29,t43,t48
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [6]:
print(storeList['list'][0])

{'seq': 0, 'sido_cd': None, 'sido_nm': None, 'gugun_cd': None, 'gugun_nm': None, 'code_order': None, 'view_yn': None, 'store_num': None, 'sido': None, 'gugun': None, 'address': None, 'new_img_nm': None, 'p_pro_seq': 0, 'p_view_yn': None, 'p_sido_cd': '', 'p_gugun_cd': '', 'p_store_nm': None, 'p_theme_cd': None, 'p_wireless_yn': None, 'p_smoking_yn': None, 'p_book_yn': None, 'p_music_yn': None, 'p_terrace_yn': None, 'p_table_yn': None, 'p_takeout_yn': None, 'p_parking_yn': None, 'p_dollar_assent': None, 'p_card_recharge': None, 'p_subway_yn': None, 'stb_store_file_renew': None, 'stb_store_theme_renew': None, 'stb_store_time_renew': None, 'stb_store_lsm': None, 's_code': '1509', 's_name': '역삼아레나빌딩', 'tel': '1522-3232', 'fax': '02-568-3763', 'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0101', 'gugun_name': '강남구', 'addr': '서울특별시 강남구 역삼동 721-13 아레나빌딩', 'park_info': None, 'new_state': None, 'theme_state': 'T05@T08@T16@T17@T20@T21@T30@T36@T52@P80@P90', 'new_bool': 0, 'search_text': ''

In [8]:
# 작업에 필요한 컬럼 몇 가지를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name => 지점이름
# sido_code => 시도코드
# sido_name => 시도이름
# gugun_code => 구군코드
# gugun_name => 구군이름
# doro_address => 도로명주소
# lat => 위도
# lot => 경도

star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map.head()

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.514132,127.020563
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,스탈릿대치R,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583


In [9]:
star_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [11]:
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
star_df_map.dtypes

s_name           object
sido_code        object
sido_name        object
gugun_code       object
gugun_name       object
doro_address     object
lat             float64
lot             float64
dtype: object

In [12]:
# 종각점
star_df_map[star_df_map['s_name'] == '종각']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
418,종각,01,서울,0118,종로구,서울특별시 종로구 종로 64 (종로2가),37.569918,126.984528


In [13]:
# 종로관철점
star_df_map[star_df_map['s_name'] == '종로관철']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
420,종로관철,01,서울,0118,종로구,"서울특별시 종로구 종로12길 21, 2층 (관철동)",37.569058,126.986013


In [16]:
# folium 모듈의 Map() 함수로 location 속성으로 지정한 위치를 중심으로 하는 zoom_start 옵션으로 지정한 배율을 가지는 지도를 
# 만든다.
star_map = folium.Map(location=[37.57000277059541, 126.9836630540502], zoom_start=18)
# folium 모듈을 Marker() 함수로 location 속성으로 지정한 위치에 마커를 만들고 add_to() 함수로 지도에 추가한다.
# folium 모듈의 Popup() 함수로 마커를 클릭했을 때 표시할 팝업 메시지와 max_width 속성으로 팝업 메시지가 표시되는 창의 최대
# 크기를 지정할 수 있다.
popup = folium.Popup('종각점', max_width=200)
folium.Marker(location=[37.569918, 126.984528], popup=popup).add_to(star_map)
popup = folium.Popup('종로관철점', max_width=200)
folium.Marker(location=[37.569058, 126.986013], popup=popup).add_to(star_map)
star_map.save('./output/star1.html')
star_map

In [18]:
# 서울특별시에 위치한 스타벅스 전 지점의 위치를 지도위에 표시한다.
# 지도를 작성할 때 지도의 중심으로 설정할 위치가 애매할 경우 지도위에 표시할 모든 스타벅스 지점의 위도와 경도의 평균 위치를
# 중심으로 하는 지도를 만들면 된다.
star_map = folium.Map(location=[star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start=12)
# iterrows() 함수는 데이터프레임에 저장된 데이터의 인덱스와 그 인덱스에 해당되는 데이터를 얻어온다.
for index, data in star_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(star_map)
star_map.save('./output/star2.html')
star_map

In [19]:
# 강남구 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=OINSMG6A8S'
request = requests.post(targetSite, data={
    'ins_lat': 37.5108295,
    'ins_lng': 127.0292881,
    'p_sido_cd': '',
    'p_gugun_cd': '0101', 
    'in_biz_cd': '',
    'iend': 1700,
    'set_date': ''
})
storeList = request.json()
star_df = json_normalize(storeList, 'list')
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)

star_map = folium.Map(location=[star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start=14)
for index, data in star_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(star_map)
star_map.save('./output/star3.html')
star_map

In [20]:
# 경기도 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=OINSMG6A8S'
request = requests.post(targetSite, data={
    'ins_lat': 37.5108295,
    'ins_lng': 127.0292881,
    'p_sido_cd': '08',
    'p_gugun_cd': '', 
    'in_biz_cd': '',
    'iend': 1700,
    'set_date': ''
})
storeList = request.json()
star_df = json_normalize(storeList, 'list')
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)

star_map = folium.Map(location=[star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start=10)
for index, data in star_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(star_map)
star_map.save('./output/star4.html')
star_map

In [21]:
# 제주도 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=OINSMG6A8S'
request = requests.post(targetSite, data={
    'ins_lat': 37.5108295,
    'ins_lng': 127.0292881,
    'p_sido_cd': '16',
    'p_gugun_cd': '', 
    'in_biz_cd': '',
    'iend': 1700,
    'set_date': ''
})
storeList = request.json()
star_df = json_normalize(storeList, 'list')
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)

star_map = folium.Map(location=[star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start=11)
for index, data in star_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(star_map)
star_map.save('./output/star5.html')
star_map

In [22]:
# 양주시 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=OINSMG6A8S'
request = requests.post(targetSite, data={
    'ins_lat': 37.5108295,
    'ins_lng': 127.0292881,
    'p_sido_cd': '',
    'p_gugun_cd': '0821', 
    'in_biz_cd': '',
    'iend': 1700,
    'set_date': ''
})
storeList = request.json()
star_df = json_normalize(storeList, 'list')
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)

star_map = folium.Map(location=[star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start=12)
for index, data in star_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(star_map)
star_map.save('./output/star6.html')
star_map